In [ ]:
import librosa
import os
from time import sleep

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
from scipy.signal import stft
import numpy as np
from matplotlib import gridspec

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from multiprocessing import Process, Queue

In [ ]:
'''
https://stackoverflow.com/questions/36074455/python-matplotlib-with-a-line-color-gradient-and-colorbar
https://stackoverflow.com/questions/31940285/plot-a-polar-color-wheel-based-on-a-colormap-using-python-matplotlib
'''

In [ ]:
def draw_rainbow_polar(fig_name, i, data, l_bound, u_bound, cmap='hsv'):
    y = np.concatenate([data, data[:1]])
    
    steps = np.arange(0, 2*np.pi, 2*np.pi/len(data))
    steps = np.concatenate([steps, [2*np.pi]])

    norm = mpl.colors.Normalize(0.0, 2*np.pi)

    ax = plt.subplot(1, 1, 1, polar=True)
    ax.set_facecolor('whitesmoke')

    points = np.array([steps, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = mcoll.LineCollection(segments, 
                              array=steps, 
                              cmap=cmap, 
                              norm=norm,
                              linewidth=1, 
                              alpha=1.0)

    ax.add_collection(lc)
    plt.title(f'Step {i:04d}')
    plt.ylim([l_bound, u_bound])
    ax.get_yaxis().set_visible(True)
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    
    plt.savefig(os.path.join('fig', fig_name))
    plt.close()

In [ ]:
path = 'music.wav'
fs = 44100
block_length = 1
frame_length = int(fs * 0.2)
hop_length = frame_length

# mel_banks = librosa.filters.mel(sr=fs, n_fft=0.1*fs, n_mels=2206)
# mel_banks = librosa.filters.mel(sr=fs, n_fft=0.1*fs, n_mels=80)
# mel_banks.shape
# (80, 2206)

In [ ]:
fft_list = list()

stream = librosa.core.stream(path, block_length, frame_length, hop_length)

for i, block in tqdm(enumerate(stream)):
    if len(block) < frame_length:
        continue
    f, t, Zxx = stft(block, fs=fs, nperseg=0.1*fs)
    Sxx = np.abs(Zxx)
    spectrogam = 20 * np.log10(np.maximum(Sxx, 1e-8))
    spectrogam = (spectrogam + 160) / 160

    fft = np.mean(spectrogam, axis=1)
    
    fft_list.append(fft)
    
fft_mean = np.mean(fft_list, axis=0)
fft_std = np.std(fft_list, axis=0)

In [ ]:
stream = librosa.core.stream(path, block_length, frame_length, hop_length)

for i, block in tqdm(enumerate(stream)):
    if len(block) < frame_length:
        continue
    f, t, Zxx = stft(block, fs=fs, nperseg=0.1*fs)
    Sxx = np.abs(Zxx)
    spectrogam = 20 * np.log10(np.maximum(Sxx, 1e-8))
    spectrogam = (spectrogam + 160) / 160
    fft = np.mean(spectrogam, axis=1)
    
    fft_diff = fft - fft_mean
    draw_rainbow_polar(f'{i:04d}.png', i, fft_diff, -1.0, 0.5)
    
    fft_diff_norm = fft_diff/fft_std
    draw_rainbow_polar(f'norm_{i:04d}.png', i, fft_diff_norm, -4.0, 4.0)
            

In [ ]:
stream = librosa.core.stream(path, block_length, frame_length, hop_length)

for i, block in tqdm(enumerate(stream)):
#     if i < 50:
#         continue
    f, t, Zxx = stft(block, fs=fs, nperseg=0.1*fs)
    Sxx = np.abs(Zxx)
    spectrogam = 20 * np.log10(np.maximum(Sxx, 1e-8))
#     print(len(f), len(t), Zxx.shape)
    spectrogam = (spectrogam + 160) / 160
#     fig, axes = plt.subplots(1, 5)
#     im = axes[0:4].imshow(spectrogam, origin='lower', aspect='auto')
#     plt.colorbar(im)
#     plt.show()

#     fig = plt.figure(figsize=(8, 8)) 
#     grid_spec_dict = {
# #                       'nrows': 1, # row 몇 개 
# #                       'ncols': 2, # col 몇 개 
#                       'height_ratios': [1], 
#                       'width_ratios': [3, 12]
        
#     }
#     gs = gridspec.GridSpec(**grid_spec_dict)
    
#     axes = list(map(plt.subplot, gs))
#     fft = np.mean(spectrogam, axis=1)
#     axes[0].plot(fft, range(len(fft)))
#     im = axes[1].imshow(spectrogam, origin='lower', aspect='auto')
#     plt.colorbar(im)
#     plt.show()

#     fig, axes = plt.subplots(1, 2, sharey=True, gridspec_kw=grid_spec_dict)
#     fft = np.mean(spectrogam, axis=1)
#     axes[0].plot(fft - fft_mean, range(len(fft)))
#     axes[0].set_xlim([0, 1])
#     im = axes[1].imshow(spectrogam, origin='lower', aspect='auto')
#     plt.colorbar(im)
#     plt.tight_layout()
#     plt.show()
    
#     fig, axes = plt.subplots(1, 2, sharey=True, gridspec_kw=grid_spec_dict)
#     fft = np.mean(spectrogam, axis=1)
#     axes[0].plot(fft, range(len(fft)))
#     axes[0].set_xlim([0, 1])
#     im = axes[1].imshow(spectrogam, origin='lower', aspect='auto')
#     plt.colorbar(im)
#     plt.tight_layout()
#     plt.show()
        
    pass

In [ ]:
## steps = np.arange(0, 2*np.pi, 2*np.pi/len(fft_mean))
# plt.figure()
plt.polar(steps, fft - fft_mean, '')
plt.title('Step')
plt.ylim([-1.0, 0.5])
ax = plt.gca()
ax.get_yaxis().set_visible(True)
# ax.set_xticks([])
# ax.set_yticks([])
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
# plt.axis('off')
# plt.xticks()
# plt.yticks()
plt.show()